# Delta Lake - Cidades Brasileiras

Este notebook demonstra a criação, leitura e manipulação de uma tabela Delta usando o dataset `cidades_brasileiras.csv`.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
from delta import configure_spark_with_delta_pip

builder = SparkSession.builder     .appName("Delta Lake - Cidades")     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("cidade", StringType(), True),
    StructField("estado", StringType(), True),
    StructField("sigla", StringType(), True),
    StructField("ibge", IntegerType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True)
])

In [ ]:
df = spark.read.csv("../data/cidades_brasileiras.csv", header=True, schema=schema)
df.show(5)

In [ ]:
df.write.format("delta").mode("overwrite").save("../output/delta/cidades")
spark.sql("DROP TABLE IF EXISTS cidades_delta")
spark.sql("CREATE TABLE cidades_delta USING DELTA LOCATION '../output/delta/cidades'")

In [ ]:
from pyspark.sql import Row

nova_cidade = [Row(id=9999, cidade="Nova Esperança", estado="Paraná", sigla="PR", ibge=9999999, latitude=-23.5, longitude=-51.5)]
nova_df = spark.createDataFrame(nova_cidade)
nova_df.write.format("delta").mode("append").save("../output/delta/cidades")

In [ ]:
spark.sql("UPDATE cidades_delta SET cidade = 'Cidade Atualizada' WHERE id = 9999")

In [ ]:
spark.sql("DELETE FROM cidades_delta WHERE id = 9999")

In [ ]:
df_final = spark.read.format("delta").load("../output/delta/cidades")
df_final.show(5)